## Example: Converting a Segmentation Stack for Use with TrackGardener

This notebook provides a practical example of converting a segmentation stack - where each object’s value corresponds to its unique TrackID - to a TrackGardener database using the configuration file `Fluo-N2DL-HeLa-01_config_trackID.yaml`.

In this workflow, relationships between parent and offspring tracks are inferred based on their proximity in time and space. This approach is generally error-prone and not recommended for most applications. For more accurate lineage relationships, use an explicit graph structure; see labels_and_geff_to_TrackGardener for details of importing the graph structure from the `geff` format.

You can reuse the configuration file demonstrated here with the TrackGardener plugin itself. Just be sure to specify the correct paths to your imaging dataset and the TrackGardener database within the config file. Using absolute (rather than relative as for the purpose of this example) paths is the safest choice to ensure your files are correctly located.

In [1]:
import dask.array as da
import yaml

from track_gardener.converters import (
    assign_tracks_relationships,
    trackID_array_to_TG,
)

In [2]:
# pathways to the segmentation stack and configuration file

segm_path = './Fluo-N2DL-HeLa-01_segm.zarr/labels'
config_path = './Fluo-N2DL-HeLa-01_config_trackID.yaml'

In [3]:
# read in configuration file
with open(config_path) as file:
    config = yaml.safe_load(file)

In [4]:
# read segmentation array
trackID_array = da.from_zarr(segm_path)

In [5]:
# convert and save to Track Gardener database
trackID_array_to_TG(trackID_array, config)

In [6]:
# guess parent-offspring relationships by space-time proximity
db_path = config['database']['path']
assign_tracks_relationships(db_path=db_path, parent_radius=100)

### Validate the resulting database

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from track_gardener.db.db_validate import run_tracking_db_checks

In [8]:
engine = create_engine(f'sqlite:///{db_path}')
Session = sessionmaker(bind=engine)

In [9]:
with Session() as session:
    errors = run_tracking_db_checks(session)

errors

[]